In [ ]:
### Opinion Review 데이터 세트를 이용한 문서 군집화 수행하기
* 문서군집화는 동일한 군집에 속하는 문서를 같은 카테고리 소속으로 분류, 텍스트 분류기반의 문서분류와 유사
* 문서군집화는 비지도, 텍스트 분류기반 문서분류는 지도(결정 카테고리 값을 가진 학습데이터 세트 필요)

In [ ]:
#### UCI 머신러닝 리포지토리 Opinion Review 데이터 세트
* /OpinosisDataset1.0/topic   ==> uci Opinosis Opinion
    - topic는 51개의 텍스트 파일로 구성,
    - 각 파일은 Tripadvisor(호텔), Edmunds.com(자동차), Amazon.com(전자제품) 사이트에서 가져온 리뷰문서
       ==> 파일이름을 보면 어떠한 제품/서비스에 대한 리뷰인지 알수 있음
    - 각 문서는 100개 정도의 문장
* 원래 토픽 모델링 논문용인데 문서 군집화로 이용해 각 리뷰 분류

In [3]:
import pandas as pd
import glob ,os

# 아래는 제 컴퓨터에서 압축 파일을 풀어 놓은 디렉토리이니, 여러분의 디렉토리를 설정해 주십시요  
path = r'/home/jovyan/work/ml-definitive-guide/8장/OpinosisDataset1.0/topics'                     
# path로 지정한 디렉토리 밑에 있는 모든 .data 파일들의 파일명을 리스트로 취합
all_files = glob.glob(os.path.join(path, "*.data"))    
filename_list = []
opinion_text = []

# 개별 파일들의 파일명은 filename_list 리스트로 취합, 
# 개별 파일들의 파일내용은 DataFrame로딩 후 다시 string으로 변환하여 opinion_text 리스트로 취합 
for file_ in all_files:
    #print(file_) # 파일을 읽어들일때 내림차순이 아니고 순서가 없음
    # 개별 파일을 읽어서 DataFrame으로 생성 
    df = pd.read_table(file_,index_col=None, header=0,encoding='latin1')
    
    # 절대경로로 주어진 file 명을 가공. 만일 Linux에서 수행시에는 아래 \\를 / 변경. 맨 마지막 .data 확장자도 제거
    filename_ = file_.split('/')[-1]
    filename = filename_.split('.')[0]

    #파일명 리스트와 파일내용 리스트에 파일명과 파일 내용을 추가. 
    filename_list.append(filename)
    opinion_text.append(df.to_string())

# 파일명 리스트와 파일내용 리스트를  DataFrame으로 생성
document_df = pd.DataFrame({'filename':filename_list, 'opinion_text':opinion_text})
document_df.head()

,filename,opinion_text
0,bathroom_bestwestern_hotel_sfo,"The room was not overly big, but clean and..."
1,mileage_honda_accord_2008,"It's quiet, get good gas mileage and look..."
2,quality_toyota_camry_2007,I previously owned a Toyota 4Runner which ...
3,service_swissotel_hotel_chicago,Mediocre room and service for a very extr...
4,buttons_amazon_kindle,I thought it would be fitting to christen ...


In [ ]:
### TF-IDF 형태로 피처 벡터화

In [4]:
from nltk.stem import WordNetLemmatizer
import nltk
import string

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
lemmar = WordNetLemmatizer()

def LemTokens(tokens):
    return [lemmar.lemmatize(token) for token in tokens]

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english' , \
                             ngram_range=(1,2), min_df=0.05, max_df=0.85 )

#opinion_text 컬럼값으로 feature vectorization 수행
feature_vect = tfidf_vect.fit_transform(document_df['opinion_text'])


/opt/conda/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [ ]:
### 문서별 텍스트가 TF-IDF 변환된 피처 벡터화 행렬 데이터에 대해 군집화를 수행해 어떤 문서끼리 군집되는지 확인
* 문서유형 : 전자제품, 자동차, 호텔
    - 전자제품 : 네비게이션, 아이팟, 킨들, 랩탑컴퓨터 등

In [6]:
from sklearn.cluster import KMeans

# 5개 집합으로 군집화 수행. 예제를 위해 동일한 클러스터링 결과 도출용 random_state=0 
km_cluster = KMeans(n_clusters=5, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_
cluster_centers = km_cluster.cluster_centers_

# 각 데이터별로 할당된 군집의 레이블을 파일명과 파일내용을 가지고 있는 document_df DataFrame에 cluster_label 칼럼으로 추가 저장
# 각 파일명은 의견리뷰에 대한 주제, 군집이 각 주제별로 유사한 형태로 잘 구성되었는지 알아보자

In [7]:
document_df['cluster_label'] = cluster_label
document_df.head()

,filename,opinion_text,cluster_label
0,bathroom_bestwestern_hotel_sfo,"The room was not overly big, but clean and...",4
1,mileage_honda_accord_2008,"It's quiet, get good gas mileage and look...",0
2,quality_toyota_camry_2007,I previously owned a Toyota 4Runner which ...,0
3,service_swissotel_hotel_chicago,Mediocre room and service for a very extr...,1
4,buttons_amazon_kindle,I thought it would be fitting to christen ...,2


In [8]:
# 판다스 sort_values(by=정렬칼럼명) 데이터 정렬 -> cluster_label별 어떤 파일명으로 매칭됐는지 군집화 결과 확인

In [9]:
document_df[document_df['cluster_label']==0].sort_values(by='filename')

,filename,opinion_text,cluster_label
37,comfort_honda_accord_2008,"Drivers seat not comfortable, the car its...",0
16,comfort_toyota_camry_2007,Ride seems comfortable and gas mileage fa...,0
19,gas_mileage_toyota_camry_2007,Ride seems comfortable and gas mileage fa...,0
34,interior_honda_accord_2008,I love the new body style and the interior...,0
31,interior_toyota_camry_2007,"First of all, the interior has way too ma...",0
1,mileage_honda_accord_2008,"It's quiet, get good gas mileage and look...",0
8,performance_honda_accord_2008,"Very happy with my 08 Accord, performance i...",0
2,quality_toyota_camry_2007,I previously owned a Toyota 4Runner which ...,0
14,seats_honda_accord_2008,Front seats are very uncomfor...,0
13,sound_ipod_nano_8gb,headphone jack i got a clear case for it a...,0


In [10]:
document_df[document_df['cluster_label']==1].sort_values(by='filename')

,filename,opinion_text,cluster_label
25,food_holiday_inn_london,The room was packed to capacity with queu...,1
7,food_swissotel_chicago,The food for our event was deli...,1
43,free_bestwestern_hotel_sfo,The wine reception is a great idea as it i...,1
15,location_bestwestern_hotel_sfo,"Good Value good location , ideal ...",1
35,location_holiday_inn_london,Great location for tube and we crammed in...,1
9,parking_bestwestern_hotel_sfo,Parking was expensive but I think this is ...,1
38,price_amazon_kindle,"If a case was included, as with the Kindle...",1
50,price_holiday_inn_london,"All in all, a normal chain hotel on a nice...",1
49,service_bestwestern_hotel_sfo,Both of us having worked in tourism for o...,1
48,service_holiday_inn_london,"not customer, oriented hotelvery low servi...",1


In [11]:
document_df[document_df['cluster_label']==2].sort_values(by='filename')

,filename,opinion_text,cluster_label
40,battery-life_amazon_kindle,After I plugged it in to my USB hub on my ...,2
47,battery-life_ipod_nano_8gb,short battery life I moved up from a...,2
29,battery-life_netbook_1005ha,"6GHz 533FSB cpu, glossy display, 3, Cell 2...",2
4,buttons_amazon_kindle,I thought it would be fitting to christen ...,2
21,eyesight-issues_amazon_kindle,It feels as easy to read as the K1 but doe...,2
45,fonts_amazon_kindle,Being able to change the font sizes is aw...,2
27,keyboard_netbook_1005ha,", I think the new keyboard rivals the gre...",2
36,navigation_amazon_kindle,"In fact, the entire navigation structure h...",2
23,performance_netbook_1005ha,The Eee Super Hybrid Engine utility lets u...,2
39,screen_garmin_nuvi_255W_gps,It is easy to read and when touching the...,2


In [12]:
document_df[document_df['cluster_label']==3].sort_values(by='filename')

,filename,opinion_text,cluster_label
22,accuracy_garmin_nuvi_255W_gps,", and is very, very acc...",3
6,directions_garmin_nuvi_255W_gps,You also get upscale features like spoken ...,3
46,display_garmin_nuvi_255W_gps,3 quot widescreen display was a ...,3
17,features_windows7,"I had to uninstall anti, virus and selecte...",3
5,satellite_garmin_nuvi_255W_gps,It's fast to acquire satel...,3
10,speed_garmin_nuvi_255W_gps,Another feature on the 255w is a display of...,3
44,updates_garmin_nuvi_255W_gps,Another thing to consider was that I paid $...,3


In [13]:
document_df[document_df['cluster_label']==4].sort_values(by='filename')

,filename,opinion_text,cluster_label
0,bathroom_bestwestern_hotel_sfo,"The room was not overly big, but clean and...",4
33,room_holiday_inn_london,"We arrived at 23,30 hours and they could n...",4
41,rooms_bestwestern_hotel_sfo,"Great Location , Nice Rooms , Helpless...",4
26,rooms_swissotel_chicago,The Swissotel is one of our favorite hotel...,4


In [ ]:
### 중심개수를 5개에서 3개로 

In [14]:
from sklearn.cluster import KMeans

# 3개의 집합으로 군집화 
km_cluster = KMeans(n_clusters=3, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_


# 소속 클러스터를 cluster_label 컬럼으로 할당하고 cluster_label 값으로 정렬
document_df['cluster_label'] = cluster_label
document_df.sort_values(by='cluster_label')

,filename,opinion_text,cluster_label
34,interior_honda_accord_2008,I love the new body style and the interior...,0
1,mileage_honda_accord_2008,"It's quiet, get good gas mileage and look...",0
2,quality_toyota_camry_2007,I previously owned a Toyota 4Runner which ...,0
5,satellite_garmin_nuvi_255W_gps,It's fast to acquire satel...,0
16,comfort_toyota_camry_2007,Ride seems comfortable and gas mileage fa...,0
37,comfort_honda_accord_2008,"Drivers seat not comfortable, the car its...",0
8,performance_honda_accord_2008,"Very happy with my 08 Accord, performance i...",0
31,interior_toyota_camry_2007,"First of all, the interior has way too ma...",0
19,gas_mileage_toyota_camry_2007,Ride seems comfortable and gas mileage fa...,0
11,transmission_toyota_camry_2007,"After slowing down, transmission has to b...",0


In [ ]:
### 군집(Cluster)별 핵심 단어 추출하기
* KMeans 객체의 clusters_centers_ 속성
    - 각 군집을 구성하는 단어 피처가 군집의 중심을 기준으로 얼마나 가깝게 위치해 있는지 
    - 배열 값 : 행은 개별군집을, 열은 개별 피처를 의미  ==> 개별군집내 상대위치 표현한 좌표값
        + cluster_centers[0,1] : 0번 군집의 2번째 피처값

In [15]:
# 앞 예제에서 군집3개를 생성한 KMeans객체인 Km_cluster에서 cluster_centers_ 속성값 확인
cluster_centers = km_cluster.cluster_centers_
print('cluster_centers shape :',cluster_centers.shape)
print(cluster_centers)

# 군집 3개,  word 피처가 2409개로 구성되었음
# 각 행의 배열값은 각 군집내의 2409개의 피처의 위치가 개별중심과 얼마나 가까운가를 상대값으로 표현
    # 0에서 1까지의 값을 가질 수 있으며, 1에 가까울수록 중심과 가까운 값을 의미

cluster_centers shape : (3, 2409)
[[0.00319471 0.00114424 0.         ... 0.         0.         0.        ]
 [0.         0.00170335 0.0025537  ... 0.0032582  0.00349413 0.        ]
 [0.01811434 0.         0.         ... 0.         0.         0.00512036]]


In [ ]:
### cluster_centers_ 속성값(ndarray)을 이용해 각 군집별 핵심단어 추출
* ndarray의 argsort()[::-1]이용하면 배열내 큰 값을 가진 인덱스 반환 (정렬이 아님)
    - 핵심단어 피처의 이름을 추출하기 위해서
    ==> def get_cluster_details()

In [16]:
# 군집별 top n 핵심단어, 그 단어의 중심 위치 상대값, 대상 파일명들을 반환함. 
def get_cluster_details(cluster_model, cluster_data, feature_names, clusters_num, top_n_features=10):
    cluster_details = {}
    
    # cluster_centers array 의 값이 큰 순으로 정렬된 index 값을 반환
    # 군집 중심점(centroid)별 할당된 word 피처들의 거리값이 큰 순으로 값을 구하기 위함.  
    centroid_feature_ordered_ind = cluster_model.cluster_centers_.argsort()[:,::-1]
    
    #개별 군집별로 iteration하면서 핵심단어, 그 단어의 중심 위치 상대값, 대상 파일명 입력
    for cluster_num in range(clusters_num):
        # 개별 군집별 정보를 담을 데이터 초기화. # 이중 딕셔너리 
        cluster_details[cluster_num] = {}
        cluster_details[cluster_num]['cluster'] = cluster_num  # dict[key][key]
        
        # cluster_centers_.argsort()[:,::-1] 로 구한 index 를 이용하여 top n 피처 단어를 구함. 
        top_feature_indexes = centroid_feature_ordered_ind[cluster_num, :top_n_features]
        top_features = [ feature_names[ind] for ind in top_feature_indexes ]
        
        # top_feature_indexes를 이용해 해당 피처 단어의 중심 위치 상댓값 구함 
        top_feature_values = cluster_model.cluster_centers_[cluster_num, top_feature_indexes].tolist()
        
        # cluster_details 딕셔너리 객체에 개별 군집별 핵심 단어와 중심위치 상대값, 그리고 해당 파일명 입력
        cluster_details[cluster_num]['top_features'] = top_features
        cluster_details[cluster_num]['top_features_value'] = top_feature_values
        
        filenames = cluster_data[cluster_data['cluster_label'] == cluster_num]['filename']
        filenames = filenames.values.tolist()
        cluster_details[cluster_num]['filenames'] = filenames
        
    return cluster_details

# 딕셔너리 반환
# 개별군집번호cluster,핵심단어top_features,핵심단어중심위치상대값top_features_value,파일명filenames 속성값 정보


In [17]:

def print_cluster_details(cluster_details):
    for cluster_num, cluster_detail in cluster_details.items():
        print('####### Cluster {0}'.format(cluster_num))
        print('Top features:', cluster_detail['top_features'])
        print('Reviews 파일명 :',cluster_detail['filenames'][:7])
        print('==================================================')


In [24]:
feature_names = tfidf_vect.get_feature_names()

cluster_details = get_cluster_details(cluster_model=km_cluster, cluster_data=document_df,\
                                  feature_names=feature_names, clusters_num=3, top_n_features=10 )
# 인자 : 군집화 모델, cluster_data(DafaFrame),
# 인자 : feature_names (인덱스를 받아서 해당 feature 핵심단어 word를 추출한 리스트), clusters_num(군집개수)=3

In [25]:
import pprint
pprint.pprint(cluster_details)

{0: {'cluster': 0,
     'filenames': ['mileage_honda_accord_2008',
                   'quality_toyota_camry_2007',
                   'satellite_garmin_nuvi_255W_gps',
                   'performance_honda_accord_2008',
                   'transmission_toyota_camry_2007',
                   'seats_honda_accord_2008',
                   'comfort_toyota_camry_2007',
                   'gas_mileage_toyota_camry_2007',
                   'interior_toyota_camry_2007',
                   'speed_windows7',
                   'interior_honda_accord_2008',
                   'comfort_honda_accord_2008'],
     'top_features': ['interior',
                      'seat',
                      'mileage',
                      'comfortable',
                      'car',
                      'gas',
                      'transmission',
                      'gas mileage',
                      'satellite',
                      'ride'],
     'top_features_value': [0.19732815989869354,
               

In [26]:
print_cluster_details(cluster_details)

####### Cluster 0
Top features: ['interior', 'seat', 'mileage', 'comfortable', 'car', 'gas', 'transmission', 'gas mileage', 'satellite', 'ride']
Reviews 파일명 : ['mileage_honda_accord_2008', 'quality_toyota_camry_2007', 'satellite_garmin_nuvi_255W_gps', 'performance_honda_accord_2008', 'transmission_toyota_camry_2007', 'seats_honda_accord_2008', 'comfort_toyota_camry_2007']
####### Cluster 1
Top features: ['room', 'hotel', 'service', 'location', 'staff', 'food', 'clean', 'bathroom', 'parking', 'room wa']
Reviews 파일명 : ['bathroom_bestwestern_hotel_sfo', 'service_swissotel_hotel_chicago', 'food_swissotel_chicago', 'parking_bestwestern_hotel_sfo', 'staff_swissotel_chicago', 'location_bestwestern_hotel_sfo', 'staff_bestwestern_hotel_sfo']
####### Cluster 2
Top features: ['screen', 'battery', 'life', 'battery life', 'keyboard', 'kindle', 'size', 'button', 'voice', 'easy']
Reviews 파일명 : ['buttons_amazon_kindle', 'directions_garmin_nuvi_255W_gps', 'speed_garmin_nuvi_255W_gps', 'sound_ipod_nano_